<a href="https://colab.research.google.com/github/youssef13494/Twitter_Sentiment_Analysis-/blob/main/Copy_of_Twitter_Sentiment_Analysis_using_CNN_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sentiment140:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2477%2F4140%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240712%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240712T221250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3a2c00154f12d2b59358cc5e64a43f1acdd0773a1250abf82e1b64be6e20085de4556daf5bb7699dabcdbb117e261b13642bed528f3e84ad2c205e7a6b670e1bcdca7c1d489bbfcef363b2d0380b37d4a810225f0329066cf69609c398b84a96abd101f51edc32de66275caddada15a2c34837b8a130bdb1a5ce6079ba85caed578f16ccdc08d954cec545001f8e4ad65b0269fc37795af570d6aeacd51c1dc7a0c0e279e7ab22767b2f5cf1d38e6ae5ba22a8def9a61b8ad4ed1afbc48b6377597215c089fdbe81445055d91ce48ac1158cc0b4669a2b3a90c6851c0412eca1d666ddcbb1e2eccb4c28cff93a837beb252fe9b62ecf778a3466b3b29a044fbb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 84855679 bytes downloaded
Downloaded and uncompressed: sentiment140
Data source import complete.


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 159.2 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.1-py3-none-any.whl (238 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246766 sha256=2241b92f9d786709ce7a5a5c313e9a1b62cc1566bf3998b03064532ec2a56a02
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import fasttext
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout, Dense


In [ ]:
# Load the sentiment140 dataset
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', header=None,
                 names=["target", "ids", "date", "flag", "user", "text"])

# Drop unnecessary columns
df = df.drop(columns=["ids", "date", "flag", "user"])


In [ ]:
df['target'].unique()

array([0, 4])

In [ ]:
df=df[:1200000]

In [ ]:
# Map sentiment labels to 0 (negative), , and 1 (positive)
df['target'] = df['target'].map({0: 0, 4: 1})

<ipython-input-7-a66b328d45bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['target'].map({0: 0, 4: 1})


In [ ]:
df = df.sample(frac=1, random_state=42)
df

,target,text
372605,0,is studying for mid year exams!!!
551204,0,I was inches from squishing a kitty. I wouldn'...
240320,0,Is pissed forgot to set my alarm for church I ...
1047361,1,woo hoo i have finally found a number for hsbc...
555362,0,Is ready to go home and sleep...i think i migh...
...,...,...
110268,0,im starting my many hours of work now
259178,0,this song's middle change just doesn't want to...
131932,0,@ProudGamerTweet I rather average 32370
671155,0,Pickin up @misstinayao waitin on @sadittysash ...


In [ ]:
# Split data into train and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_data

,target,text
454113,0,talkin' to papa &amp; told me to call korean a...
177274,0,"Also, why are paracetamol so hard to swallow? ..."
2734,0,ARGH...Firefox has just removed a Bookmarks fo...
444378,0,"@Misanthropic___ SAME!!! (well, i haven't sh..."
4662,0,sickk in class essay tomorrow last day of sc...
...,...,...
1097007,1,Shopping in bath. The sun is shining
219668,0,Im lost driving home from my cousins birthday ...
263320,0,Still getting really tired really early.... Th...
321943,0,want to go to Macau


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download the stopwords corpus
nltk.download('stopwords')

# Download the WordNet lemmatizer data
nltk.download('wordnet')

# Define English stopwords
en_stop = set(stopwords.words('english'))

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Define function to clean and preprocess text
def process_text(document):
    document = re.sub(r'\s+', ' ', document, flags=re.I)  # Remove extra white space from text
    document = re.sub(r'\W', ' ', str(document))   # Remove all the special characters from text
    document = re.sub(r'[^a-zA-Z\s]', '', document) # Remove any character that isn't alphabetical
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)     # Remove all single characters from text
    document = document.lower()    # Converting to Lowercase
    # Word tokenization
    tokens = document.split()
    lemma_txt = [lemmatizer.lemmatize(word) for word in tokens]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in en_stop]
    # Drop words
    tokens = [word for word in lemma_no_stop_txt if len(word) > 3 and not word.isdigit()]
    # Getting unique words
    indices = np.unique(tokens, return_index=True)[1]
    clean_txt = np.array(tokens)[np.sort(indices)].tolist()
    return ' '.join(clean_txt)  # Return the cleaned text as a string


In [ ]:
# Apply the process_text function to clean the text data
train_data['cleaned_text'] = train_data['text'].apply(process_text)
test_data['cleaned_text'] = test_data['text'].apply(process_text)

In [ ]:
train_data

,target,text,cleaned_text
454113,0,talkin' to papa &amp; told me to call korean a...,talkin papa told call korean airline book tick...
177274,0,"Also, why are paracetamol so hard to swallow? ...",also paracetamol hard swallow even long snappe...
2734,0,ARGH...Firefox has just removed a Bookmarks fo...,argh firefox removed bookmark folder sharepoin...
444378,0,"@Misanthropic___ SAME!!! (well, i haven't sh...",misanthropic well showered
4662,0,sickk in class essay tomorrow last day of sc...,sickk class essay tomorrow last school thurs w...
...,...,...,...
1097007,1,Shopping in bath. The sun is shining,shopping bath shining
219668,0,Im lost driving home from my cousins birthday ...,lost driving home cousin birthday party wont a...
263320,0,Still getting really tired really early.... Th...,still getting really tired early hate sick night
321943,0,want to go to Macau,want macau


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.text)

In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.cleaned_text), maxlen = 25)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.cleaned_text),maxlen = 25)

In [ ]:
print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

Training X Shape: (960000, 25)
Testing X Shape: (240000, 25)


In [ ]:
x_train.shape

(960000, 25)

In [ ]:
for i in x_train:
    if i[0]!=0:
        print('wrong')

In [ ]:
# Convert target labels to categorical
y_train = pd.get_dummies(train_data['target']).values
y_test = pd.get_dummies(test_data['target']).values

In [ ]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras.models import Model

In [ ]:
word2vec = tokenizer.word_index
V = len(word2vec)

T = 25
D = 20

i = Input((T,))
x = Embedding(V + 1, D)(i)
x = Conv1D(32, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 1, activation='relu')(x)  # Reduce kernel size to 1
x = GlobalMaxPooling1D()(x)
x = Dense(2, activation='softmax')(x)

model = Model(i, x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25)]              0         
                                                                 
 embedding (Embedding)       (None, 25, 20)            9327740   
                                                                 
 conv1d (Conv1D)             (None, 23, 32)            1952      
                                                                 
 max_pooling1d (MaxPooling1  (None, 7, 32)             0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 5, 64)             6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 1, 64)             0         
 g1D)                                                        

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
cnn_senti = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=100)


Epoch 1/5
9600/9600 [==============================] - 1090s 113ms/step - loss: 0.6298 - accuracy: 0.6681 - val_loss: 0.6274 - val_accuracy: 0.6702
Epoch 2/5
9600/9600 [==============================] - 1097s 114ms/step - loss: 0.6023 - accuracy: 0.6906 - val_loss: 0.6382 - val_accuracy: 0.6681
Epoch 3/5
9600/9600 [==============================] - 1104s 115ms/step - loss: 0.5836 - accuracy: 0.6990 - val_loss: 0.6486 - val_accuracy: 0.6661
Epoch 4/5
9600/9600 [==============================] - 1101s 115ms/step - loss: 0.5731 - accuracy: 0.7030 - val_loss: 0.6585 - val_accuracy: 0.6661
Epoch 5/5
1383/9600 [===>..........................] - ETA: 15:37 - loss: 0.5617 - accuracy: 0.7075

In [ ]:
vocab_size=V+1

In [ ]:
embedding_layer = Embedding(vocab_size,100,input_length=25,trainable=False)

In [ ]:
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D, Input, Concatenate, SimpleRNN
from keras.models import Model

# Define the model
sequence_input = Input(shape=(25,), dtype='int32')
embedding_sequences = embedding_layer(sequence_input)
x1 = Conv1D(64, 5, activation='relu')(embedding_sequences)
x1 = MaxPooling1D(pool_size=4)(x1)
x1 = SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2)(x1)
x1 = Dense(512, activation='relu')(x1)
x1 = Dense(512, activation='relu')(x1)
outputs = Dense(2, activation='sigmoid')(x1)  # Change the output size to 1 for binary classification
model_cnn = Model(sequence_input, outputs)

# Compile the model
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model_cnn.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=100, epochs=10)


In [ ]:
def predict_sentiment(model, text):
    preprocessed_text = process_text(text)
    sequence = tokenizer.texts_to_sequences([preprocessed_text])
    padded_sequence = pad_sequences(sequence, maxlen=80)
    prediction = model.predict(padded_sequence)[0][0]
    if prediction <= 0.50:
        return "Positive"
    else:
        return "Negative"

In [ ]:
# Positive sentences
positive_sentences = [
    "The weather is fantastic absolutely good gorgeous today!",
    "I'm overjoyed to spend fantastic quality time with my amazing friends and loving family.",
    "Every bite of the good food at that restaurant fantastic is a taste of pure bliss.",
    "I'm deeply good appreciative of the unwavering support and kindness from my wonderful fantastic colleagues.",
    "My vacation was an unforgettable fantastic adventure filled with joy, relaxation, and exciting discoveries."
]

# Negative sentences
negative_sentences = [
    "I'm feeling sick and tired of all the noise outside.",
    "My car broke down on the way to work, causing me to be late.",
    "The customer service at that store was terrible; they were rude and unhelpful.",
    "I'm disappointed with my exam results; I was expecting to do better.",
    "It's frustrating when my internet connection is slow, especially when I'm trying to work."
]


In [ ]:
# Sample 10 random rows from the DataFrame
random_rows = test_data.sample(n=10, random_state=1)

# Print each of the randomly selected rows
for index, row in random_rows.iterrows():
    predicted_label_firstmodel = predict_sentiment(model, row['cleaned_text'])
    true_label = 'Positive' if row['target'] == 1 else 'Negative'
    print(f"True Value is: {true_label}, Predicted Value For First Model: {predicted_label_firstmodel}")


In [ ]:
# Sample 10 random rows from the DataFrame
random_rows = test_data.sample(n=10, random_state=1)

# Print each of the randomly selected rows
for index, row in random_rows.iterrows():
    predicted_label_firstmodel = predict_sentiment(model_cnn, row['cleaned_text'])
    true_label = 'Positive' if row['target'] == 1 else 'Negative'
    print(f"True Value is: {true_label}, Predicted Value For Second Model: {predicted_label_firstmodel}")
